In [1]:
import matplotlib.pyplot as plt
from api_keys import vacation_key
from citipy import citipy
import requests
from pprint import pprint
from datetime import datetime
from pytz import timezone
import numpy as np
from glob import glob
import re
import pandas as pd
import gmaps
import json
from citipy import citipy

In [2]:
latest_file = glob('Resources/cities_weather_*.csv')[-1]
print(f'extracting from {latest_file}...')
date = re.search(r'\d+', latest_file).group()
date = datetime.strptime(date, '%Y%m%d').strftime('%m-%d-%Y')

df = pd.read_csv(latest_file)
df

extracting from Resources\cities_weather_20211023.csv...


,city,country,lat,lon,temp,humid,cloud,windsp
0,severo-yeniseyskiy,ru,60.3725,93.0408,35.29,96,100,11.95
1,huarmey,pe,-10.0681,-78.1522,62.85,86,27,7.38
2,aquiraz,br,-3.9014,-38.3911,83.32,67,69,10.40
3,saint-ambroise,ca,48.5501,-71.3324,36.45,81,90,3.85
4,makamba,bi,-4.1348,29.8040,74.75,56,38,2.51
...,...,...,...,...,...,...,...,...
495,topchanchi,in,23.9000,86.2000,82.71,50,14,0.69
496,semporna,my,4.4818,118.6112,88.00,75,99,4.38
497,kruisfontein,za,-34.0033,24.7314,61.88,54,7,12.46
498,ancud,cl,-41.8697,-73.8203,45.93,86,91,14.23


In [3]:
gmaps.configure(api_key=vacation_key)

locs = df[['lat', 'lon']]
humid = df['humid'] 
temp = df['temp']

In [4]:
fig = gmaps.figure()

# add heat layer and set max intensity to max data humidity
heat = gmaps.heatmap_layer(locs, weights=humid, dissipating=False, max_intensity=humid.max(), point_radius = 4)
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

df1 = df[(df['temp'] < 80) & (df['temp'] > 60) &\
         (df['humid'] < 50) &\
         (df['windsp'] < 15) &\
         (df['cloud'] < 5)].reset_index(drop=True)

df1

,city,country,lat,lon,temp,humid,cloud,windsp
0,heishan,cn,41.6892,122.1128,61.70,38,0,12.08
1,buraydah,sa,26.3260,43.9750,75.65,17,0,5.39
2,atar,mr,20.5169,-13.0499,76.48,21,3,7.90
3,taoudenni,ml,22.6783,-3.9836,78.26,25,0,8.37
4,tall kayf,iq,37.0833,41.0667,69.19,23,0,4.61
5,guymon,us,36.6828,-101.4816,64.35,39,1,9.22
6,muborak,uz,39.2553,65.1528,60.89,23,0,3.85
7,broken hill,au,-31.9500,141.4333,69.26,24,0,12.06
8,chaoyang,cn,41.5703,120.4586,62.31,28,0,6.53
9,seoul,kr,37.5683,126.9778,62.98,45,0,8.05


In [6]:
# ping google api for first nearby hotel
params = {
    'radius': 5000
    , 'types': 'hotel'
    , 'keyword': 'hotel'
    , 'key': vacation_key
}

for index, row in df1.iterrows():
#     print(index, list(row))

    city = row['city']
    lat = row['lat']
    lon = row['lon']
    
    params['location'] = f'{lat},{lon}'
    
    url = r'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    r = requests.get(url, params)
    dicy = r.json()
    
    try:
        hotelname = dicy['results'][0]['name']
        hotellat = dicy['results'][0]['geometry']['location']['lat']
        hotellon = dicy['results'][0]['geometry']['location']['lng']
    except:
        hotelname = ''
        hotellat = ''
        hotellon = ''
        
        print(f'hotel ERROR... {city}: ({lat}, {lon}) | {hotelname}: ({hotellat}, {hotellon})')
    else:
        df1.loc[index, 'hotelname'] = hotelname
        
        print(f'hotel found... {city}: ({lat}, {lon}) | {hotelname}: ({hotellat}, {hotellon})')

hotel found... heishan: (41.6892, 122.1128) | Ruixianglou Hotel: (41.69325200000001, 122.116772)
hotel found... buraydah: (26.326, 43.975) | Best Western Plus Buraidah: (26.3631102, 43.9302354)
hotel found... atar: (20.5169, -13.0499) | Auberge et Camping Inimi: (20.5013475, -13.0601369)
hotel ERROR... taoudenni: (22.6783, -3.9836) | : (, )
hotel ERROR... tall kayf: (37.0833, 41.0667) | : (, )
hotel found... guymon: (36.6828, -101.4816) | Hampton Inn & Suites Guymon: (36.684221, -101.469299)
hotel found... muborak: (39.2553, 65.1528) | Gostinitsa Prestizh: (39.2561346, 65.1623692)
hotel found... broken hill: (-31.95, 141.4333) | Comfort Inn Crystal: (-31.9583412, 141.467273)
hotel found... chaoyang: (41.5703, 120.4586) | Yandu International Hotel: (41.57770600000001, 120.453302)
hotel found... seoul: (37.5683, 126.9778) | Four Seasons Hotel Seoul: (37.5704525, 126.975539)


In [7]:
df1

,city,country,lat,lon,temp,humid,cloud,windsp,hotelname
0,heishan,cn,41.6892,122.1128,61.70,38,0,12.08,Ruixianglou Hotel
1,buraydah,sa,26.3260,43.9750,75.65,17,0,5.39,Best Western Plus Buraidah
2,atar,mr,20.5169,-13.0499,76.48,21,3,7.90,Auberge et Camping Inimi
3,taoudenni,ml,22.6783,-3.9836,78.26,25,0,8.37,NaN
4,tall kayf,iq,37.0833,41.0667,69.19,23,0,4.61,NaN
5,guymon,us,36.6828,-101.4816,64.35,39,1,9.22,Hampton Inn & Suites Guymon
6,muborak,uz,39.2553,65.1528,60.89,23,0,3.85,Gostinitsa Prestizh
7,broken hill,au,-31.9500,141.4333,69.26,24,0,12.06,Comfort Inn Crystal
8,chaoyang,cn,41.5703,120.4586,62.31,28,0,6.53,Yandu International Hotel
9,seoul,kr,37.5683,126.9778,62.98,45,0,8.05,Four Seasons Hotel Seoul


In [8]:
df1 = df1.dropna()
df1

,city,country,lat,lon,temp,humid,cloud,windsp,hotelname
0,heishan,cn,41.6892,122.1128,61.70,38,0,12.08,Ruixianglou Hotel
1,buraydah,sa,26.3260,43.9750,75.65,17,0,5.39,Best Western Plus Buraidah
2,atar,mr,20.5169,-13.0499,76.48,21,3,7.90,Auberge et Camping Inimi
5,guymon,us,36.6828,-101.4816,64.35,39,1,9.22,Hampton Inn & Suites Guymon
6,muborak,uz,39.2553,65.1528,60.89,23,0,3.85,Gostinitsa Prestizh
7,broken hill,au,-31.9500,141.4333,69.26,24,0,12.06,Comfort Inn Crystal
8,chaoyang,cn,41.5703,120.4586,62.31,28,0,6.53,Yandu International Hotel
9,seoul,kr,37.5683,126.9778,62.98,45,0,8.05,Four Seasons Hotel Seoul


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotelname}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotelinfo = [info_box_template.format(**row) for index, row in df1.iterrows()]
locs = df1[['lat', 'lon']]

In [10]:
mark = gmaps.marker_layer(locs, info_box_content=hotelinfo)
fig.add_layer(mark)

fig

Figure(layout=FigureLayout(height='420px'))